In [1]:
import polars as pl
import polars_ds as pds
import numpy as np

# This notebook illustrates the basic usage of this package

You need to create an environment with this package installed to run this.

# Num Extensions

In [2]:
size = 10_000
df = pl.DataFrame({
    "f": np.sin(list(range(size)))
    , "time_idx": range(size)
    , "dummy": ["a"] * (size // 2) + ["b"] * (size // 2)
    , "a": np.random.random(size = size)
    , "b": np.random.random(size = size)
    , "x1" : range(size)
    , "x2" : range(size, size + size)
    , "y": range(-size, 0)
    , "actual": np.round(np.random.random(size=size)).astype(np.int32)
    , "predicted": np.random.random(size=size)
    , "dummy_groups":["a"] * (size//2) + ["b"] * (size//2) 
})
df.head()

f,time_idx,dummy,a,b,x1,x2,y,actual,predicted,dummy_groups
f64,i64,str,f64,f64,i64,i64,i64,i32,f64,str
0.0,0,"""a""",0.136797,0.906574,0,10000,-10000,1,0.539614,"""a"""
0.841471,1,"""a""",0.103381,0.486365,1,10001,-9999,1,0.904195,"""a"""
0.909297,2,"""a""",0.31547,0.456745,2,10002,-9998,0,0.714948,"""a"""
0.14112,3,"""a""",0.876959,0.349627,3,10003,-9997,0,0.381403,"""a"""
-0.756802,4,"""a""",0.865781,0.619041,4,10004,-9996,1,0.602646,"""a"""


In [3]:
# Column-wise Jaccard Similarity. Result should be 0 as they are distinct
df.select(
    pl.col("x1").num.jaccard(pl.col("x2"))
)

x1
f64
0.0


In [4]:
# FFT. First is real part, second is complex part
# By default, this behaves the same as np's rfft, which returns a non-redundant 
# compact representation of fft output.
df.select(
    pl.col("f").num.rfft()
).head()

f
"array[f64, 2]"
"[1.939505, 0.0]"
"[1.939506, 0.000209]"
"[1.939508, 0.000418]"
"[1.939512, 0.000627]"
"[1.939518, 0.000835]"


In [5]:
# FFT. But return the full length
df.select(
    pl.col("f").num.rfft(return_full=True)
).shape

(10000, 1)

In [6]:
# Convolution (by FFT). 
# Modes: `same`, `left` (left-aligned same), `right` (right-aligned same), `valid` or `full`
# Currently slower than SciPy but provides parallelism because of Polars
df.select(
    pl.col("f").num.convolve([-1, 0, 0, 0, 1], mode = "full"),
    pl.col("a").num.convolve([-1, 0, 0, 0, 1], mode = "full"),
    pl.col("b").num.convolve([-1, 0, 0, 0, 1], mode = "full"),
).head()

f,a,b
f64,f64,f64
1.3944e-15,-0.136797,-0.906574
-0.841471,-0.103381,-0.486365
-0.909297,-0.31547,-0.456745
-0.14112,-0.876959,-0.349627
0.756802,-0.728983,0.287533


In [7]:
# Least Square (Linear Regression)
df.select(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

y
list[f64]
"[2.0, -1.0]"


In [8]:
df.select(
    pds.query_lstsq_report(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    ).alias("report")
).unnest("report")

feat_idx,coeff,std_err,t,p>|t|
u16,f64,f64,f64,f64
0,2.0,2.3854e-16,8.3842e15,0.0
1,-1.0,9.0158e-17,-1.1092e16,0.0


In [9]:
df.lazy().select(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
).collect()

y
list[f64]
"[2.0, -1.0]"


In [10]:
df.select(
    # pl.col("y").num.lstsq(pl.col("x1"), pl.col("x2"), add_bias=False).over(pl.col("dummy"))
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    ).over(pl.col("dummy"))
).head() 

coeffs
list[f64]
"[2.0, -1.0]"
"[2.0, -1.0]"
"[2.0, -1.0]"
"[2.0, -1.0]"
"[2.0, -1.0]"


In [11]:
# If you want prediction and residue instead of coefficients
df.select(
    "x1",
    "x2",
    "y",
    # This is equivalent to pds.query_lstsq, with y being the target. Linters, however, will not pick up this syntax
    (pl.col("y").num.lstsq(pl.col("x1"), pl.col("x2"), add_bias=False, return_pred=True)).alias("prediction")
).unnest("prediction").head()

x1,x2,y,pred,resid
i64,i64,i64,f64,f64
0,10000,-10000,-10000.0,0.0
1,10001,-9999,-9999.0,0.0
2,10002,-9998,-9998.0,0.0
3,10003,-9997,-9997.0,0.0
4,10004,-9996,-9996.0,0.0


In [12]:
df.group_by("dummy").agg(
    pl.col("y").num.lstsq(pl.col("x1"), pl.col("x2"), add_bias=False)
)


dummy,coeffs
str,list[f64]
"""a""","[2.0, -1.0]"
"""b""","[2.0, -1.0]"


In [13]:
# Rolling regression
df.lazy().rolling(
    index_column = pl.col("time_idx").set_sorted(),
    period = "30i",
    # offset = "-1i"
).agg(
    pds.query_lstsq(pl.col("x1"), pl.col("x2"), target = pl.col("y"), add_bias=False).alias("coefficients")
).slice(offset = 30).select(
    "time_idx",
    "coefficients",
).collect().head(10)

time_idx,coefficients
i64,list[f64]
30,"[2.0, -1.0]"
31,"[2.0, -1.0]"
32,"[2.0, -1.0]"
33,"[2.0, -1.0]"
34,"[2.0, -1.0]"
35,"[2.0, -1.0]"
36,"[2.0, -1.0]"
37,"[2.0, -1.0]"
38,"[2.0, -1.0]"


In [14]:
# Conditional Entropy, should be 0 because x1 is an ID
df.select(
    pl.col("y").num.cond_entropy(pl.col("x1"))
)

y
f64
-0.0


In [15]:
df.group_by("dummy_groups").agg(
    pl.col("actual").metric.l2_loss(pl.col("predicted")).alias("l2"),
    pl.col("actual").metric.bce(pl.col("predicted")).alias("log loss"),
    pl.col("actual").metric.binary_metrics_combo(pl.col("predicted")).alias("combo")
).unnest("combo")


dummy_groups,l2,log loss,precision,recall,f,average_precision,roc_auc
str,f64,f64,f64,f64,f64,f64,f64
"""b""",0.334028,0.990628,0.498983,0.48865,0.246881,0.508072,0.499955
"""a""",0.338878,1.025153,0.486156,0.484185,0.242584,0.48391,0.491673


# Str Extension

In [16]:
size = 100_000
df2 = pl.DataFrame({
    "sen":["Hello, world! I'm going to church."] * size,
    "word":["words", "word"] * (size //2)
})
df2.head()

sen,word
str,str
"""Hello, world! …","""words"""
"""Hello, world! …","""word"""
"""Hello, world! …","""words"""
"""Hello, world! …","""word"""
"""Hello, world! …","""words"""


In [17]:
# Tokenize
df2.select(
    pl.col("sen").str.to_lowercase().str2.tokenize().explode().unique()
)

sen
str
"""hello"""
"""world"""
"""going"""
"""to"""
"""church"""


In [18]:
df2.select(
    pl.col("sen").str.to_lowercase().str2.tokenize(stem=True).explode().unique()
)

sen
str
""""""
"""world"""
"""hello"""
"""go"""
"""church"""


In [19]:
df2.select(
    pl.col("word").str2.levenshtein("world")
).head()

word
u32
2
1
2
1
2


In [20]:
# Damerau-Levenshtein
df2.select(
    pl.col("word").str2.d_levenshtein("world")
).head()

word
u32
2
1
2
1
2


In [21]:
df2.select(
    pl.col("word").str2.levenshtein("world", return_sim = True)
).head()

word
f64
0.6
0.8
0.6
0.8
0.6


In [22]:
df2.filter(
    # This is way faster than computing ditance and then doing a filter
    pl.col("word").str2.levenshtein_filter("world", 1) # <= 1. 
).head()

sen,word
str,str
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""


In [23]:
df = pl.DataFrame({
    "word":["apple", "banana", "pineapple", "asasasas", "sasasass"],
    "other_data": [1,2,3,4,5]
})
gibberish = ["asasasa", "sasaaasss", "asdasadadfa"]

In [24]:
df.filter(
    pl.col("word").str2.similar_to_vocab(
        vocab = gibberish,
        threshold = 0.5,
        metric = "lv", # Levenshtein similarity. Other options: dleven, osa, jw
        strategy = "any" # True if the word is similar to any word in vocab. Other options: "all", "avg"
    )
)

word,other_data
str,i64
"""asasasas""",4
"""sasasass""",5


In [25]:
df.select(
    pl.col("word").str2.levenshtein("asasasa", return_sim=True).alias("asasasa"),
    pl.col("word").str2.levenshtein("sasaaasss", return_sim=True).alias("sasaaasss"),
    pl.col("word").str2.levenshtein("asdasadadfa", return_sim=True).alias("asdasadadfa"),
    pl.col("word").str2.fuzz("apples").alias("LCS based Fuzz match - apples"),
    pl.col("word").str2.osa("apples", return_sim = True).alias("Optimal String Alignment - apples"),
    pl.col("word").str2.jw("apples").alias("Jaro-Winkler - apples"),
)


asasasa,sasaaasss,asdasadadfa,LCS based Fuzz match - apples,Optimal String Alignment - apples,Jaro-Winkler - apples
f64,f64,f64,f64,f64,f64
0.142857,0.111111,0.090909,0.833333,0.833333,0.966667
0.428571,0.333333,0.272727,0.166667,0.0,0.444444
0.111111,0.111111,0.090909,0.555556,0.444444,0.5
0.875,0.666667,0.545455,0.25,0.25,0.527778
0.75,0.777778,0.454545,0.25,0.25,0.527778


# Stats Extension

In [26]:
import numpy as np

df = pl.DataFrame({
    "a": [None, None] + list(np.random.normal(size = 998))
})
df.head()

a
f64
null
null
0.10501
0.744794
0.658212


In [27]:
# Genenrate random sample, respecting null positions in reference column (pl.col("a"))
df.with_columns(
    pl.col("a").stats.sample_normal(mean = 0.5, std = 1., respect_null=True).alias("random")
).head()

a,random
f64,f64
null,null
null,null
0.10501,1.221485
0.744794,0.227725
0.658212,0.094222


In [28]:
# Genenrate random string
df.with_columns(
    pl.col("a").stats.rand_str(min_size = 1, max_size = 5, respect_null=True).alias("random_str")
).head()

a,random_str
f64,str
null,null
null,null
0.10501,"""48pH"""
0.744794,"""XY2Ry"""
0.658212,"""n4"""


In [29]:
# Genenrate fixed size random string
df.with_columns(
    pl.col("a").stats.rand_str(min_size = 5, max_size = 5, respect_null=True).alias("random_str")
).head()

a,random_str
f64,str
null,null
null,null
0.10501,"""IDGgW"""
0.744794,"""kNAwd"""
0.658212,"""G0YJD"""


In [30]:
df.with_columns(
    # Sample from normal distribution, using reference column "a" 's mean and std
    pl.col("a").stats.sample_normal().alias("test1") 
    # Sample from uniform distribution, with low = 0 and high = "a"'s max, and respect the nulls in "a"
    , pl.col("a").stats.sample_uniform(low = 0., high = None, respect_null=True).alias("test2")
).head()

a,test1,test2
f64,f64,f64
null,-0.559211,null
null,-0.263125,null
0.10501,-0.072895,3.686794
0.744794,0.114534,3.435098
0.658212,-0.100332,2.251496


In [31]:
# Genenrate 2 random sample, both normally distributed
# Run Welch's t test on them, p value should be big since they have equal mean
# Run a normality test. Again, p value should be big since they are normally distributed 

df.with_columns(
    pl.col("a").stats.sample_normal(mean = 0.5, std = 1.).alias("test1")
    , pl.col("a").stats.sample_normal(mean = 0.5, std = 2.).alias("test2")
).select(
    pl.col("test1").stats.ttest_ind(pl.col("test2"), equal_var = False).alias("t-test")
    , pl.col("test1").stats.normal_test().alias("normality_test")
).select(
    pl.col("t-test").struct.field("statistic").alias("t-tests: statistics")
    , pl.col("t-test").struct.field("pvalue").alias("t-tests: pvalue")
    , pl.col("normality_test").struct.field("statistic").alias("normality_test: statistics")
    , pl.col("normality_test").struct.field("pvalue").alias("normality_test: pvalue")
)

t-tests: statistics,t-tests: pvalue,normality_test: statistics,normality_test: pvalue
f64,f64,f64,f64
1.924642,0.054469,0.202879,0.903536


In [32]:
size = 5_000
df = pl.DataFrame({
    "market_id": range(size),
    "group1": np.random.random(size=size),
    "group2": np.random.random(size=size),
    "category_1": np.random.randint(low=0, high=5, size=size),
    "category_2":np.random.randint(low=0, high=10, size=size)
}).with_columns(
    pl.col("market_id").mod(3)
)
df.head(5)

market_id,group1,group2,category_1,category_2
i64,f64,f64,i64,i64
0,0.532197,0.491526,3,4
1,0.238987,0.042422,3,6
2,0.96231,0.825304,3,0
0,0.553353,0.238958,3,9
1,0.927642,0.736546,2,0


In [33]:
# In dataframe statistical tests!
df.select(
    pl.col("group1").stats.ttest_ind(pl.col("group2"), equal_var = True).alias("t-test"),
    pl.col("category_1").stats.chi2(pl.col("category_2")).alias("chi2-test"),
    pl.col("category_1").stats.f_test(pl.col("group1")).alias("f-test")
)

t-test,chi2-test,f-test
struct[2],struct[2],struct[2]
"{0.314334,0.753274}","{30.641615,0.721121}","{0.333509,0.85556}"


In [34]:
# Can also be done in group by context
df.group_by("market_id").agg(
    pl.col("group1").stats.ttest_ind(pl.col("group2"), equal_var = False).alias("t-test"),
    pl.col("category_1").stats.chi2(pl.col("category_2")).alias("chi2-test"),
    pl.col("category_1").stats.f_test(pl.col("group1")).alias("f-test")
)

market_id,t-test,chi2-test,f-test
i64,struct[2],struct[2],struct[2]
0,"{0.364199,0.715732}","{30.480606,0.728182}","{0.772123,0.543294}"
1,"{0.338841,0.734751}","{41.101492,0.256961}","{0.599172,0.663278}"
2,"{-0.154178,0.877479}","{33.224091,0.601321}","{0.673806,0.610149}"


# Nearest Neighbors Related Tasks

These queries can be very slow when data/dimension gets huge, even when processed in parallel.

In [35]:
import polars_ds as pds
size = 2000
df = pl.DataFrame({
    "id": range(size), 
    "val1": np.random.random(size=size), 
    "val2": np.random.random(size=size), 
    "val3": np.random.random(size=size),
    "r": np.random.random(size=size),
    "rh": np.random.random(size=size)*10,
}).with_columns(
    pl.col("id").cast(pl.UInt64)
)

In [36]:
# Get neighbor count. The point itself is always considered a neighbor to itself.
df.with_columns(
    pds.query_nb_cnt(
        0.1, # radius 
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        dist = "inf", # L Infinity distance 
        parallel = True 
    ).alias("nb_l_inf_cnt")
).head() 

id,val1,val2,val3,r,rh,nb_l_inf_cnt
u64,f64,f64,f64,f64,f64,u32
0,0.6794,0.976336,0.980353,0.053462,7.325836,6
1,0.662397,0.52761,0.995347,0.741,1.118534,13
2,0.102895,0.173097,0.167723,0.261916,7.181822,24
3,0.487158,0.55844,0.830182,0.478289,5.362583,21
4,0.575279,0.555741,0.646294,0.611059,3.163473,12


In [37]:
df.with_columns(
    pds.query_nb_cnt(
        pl.col("r"), # radius be an expression too
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        dist = "l1", # L 1 distance 
        parallel = True 
    ).alias("nb_l1_r_cnt")
).head()

id,val1,val2,val3,r,rh,nb_l1_r_cnt
u64,f64,f64,f64,f64,f64,u32
0,0.6794,0.976336,0.980353,0.053462,7.325836,2
1,0.662397,0.52761,0.995347,0.741,1.118534,487
2,0.102895,0.173097,0.167723,0.261916,7.181822,55
3,0.487158,0.55844,0.830182,0.478289,5.362583,272
4,0.575279,0.555741,0.646294,0.611059,3.163473,546


In [38]:
# Get ids of the k nearest neighbors. 
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        index = pl.col("id"),
        k = 3, 
        dist = "l2", # squared l2
        parallel = True
    ).alias("best friends")
).head() 

id,val1,val2,val3,r,rh,best friends
u64,f64,f64,f64,f64,f64,list[u64]
0,0.6794,0.976336,0.980353,0.053462,7.325836,"[0, 891, … 1934]"
1,0.662397,0.52761,0.995347,0.741,1.118534,"[1, 1195, … 138]"
2,0.102895,0.173097,0.167723,0.261916,7.181822,"[2, 1559, … 1095]"
3,0.487158,0.55844,0.830182,0.478289,5.362583,"[3, 1242, … 290]"
4,0.575279,0.555741,0.646294,0.611059,3.163473,"[4, 1443, … 1707]"


In [39]:
# Get all neighbors within radius r
# The point itself is always considered a neighbor to itself.
print(df.select(
    pl.col("id"),
    pds.query_radius_ptwise(
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        index = pl.col("id"),
        r = 0.1, 
        dist = "l2", # actually this is squared l2
        parallel = True
    ).alias("best friends"),
).with_columns( # -1 to remove the point itself
    (pl.col("best friends").list.len() - 1).alias("best friends count")
).head())

shape: (5, 3)
┌─────┬───────────────────┬────────────────────┐
│ id  ┆ best friends      ┆ best friends count │
│ --- ┆ ---               ┆ ---                │
│ u64 ┆ list[u64]         ┆ u32                │
╞═════╪═══════════════════╪════════════════════╡
│ 0   ┆ [0, 891, … 140]   ┆ 86                 │
│ 1   ┆ [1, 1195, … 1279] ┆ 138                │
│ 2   ┆ [2, 1559, … 175]  ┆ 146                │
│ 3   ┆ [3, 1242, … 1908] ┆ 238                │
│ 4   ┆ [4, 1443, … 1377] ┆ 247                │
└─────┴───────────────────┴────────────────────┘


In [40]:
# Get ids of the k nearest neighbors and distances
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        index = pl.col("id"),
        k = 3, 
        dist = "l2", # actually this is squared l2
        parallel = True,
        return_dist = True
    ).alias("best_friends_w_dist")
).unnest("best_friends_w_dist").head()

id,val1,val2,val3,r,rh,knn,radius
u64,f64,f64,f64,f64,f64,list[u64],list[f64]
0,0.6794,0.976336,0.980353,0.053462,7.325836,"[0, 891, … 1934]","[0.0, 0.001341, … 0.012567]"
1,0.662397,0.52761,0.995347,0.741,1.118534,"[1, 1195, … 138]","[0.0, 0.003094, … 0.003298]"
2,0.102895,0.173097,0.167723,0.261916,7.181822,"[2, 1559, … 1095]","[0.0, 0.001865, … 0.004513]"
3,0.487158,0.55844,0.830182,0.478289,5.362583,"[3, 1242, … 290]","[0.0, 0.000736, … 0.003501]"
4,0.575279,0.555741,0.646294,0.611059,3.163473,"[4, 1443, … 1707]","[0.0, 0.001441, … 0.00274]"


In [41]:
# Filter to only points near the given point
df.filter(
    pds.query_radius_at_pt(
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5, 0.5],
        r = 0.2,
        dist = "l2" # actually this is squared l2, so this is asking for squared l2 <= 0.2
    )
).head()

id,val1,val2,val3,r,rh
u64,f64,f64,f64,f64,f64
3,0.487158,0.55844,0.830182,0.478289,5.362583
4,0.575279,0.555741,0.646294,0.611059,3.163473
7,0.824648,0.487669,0.536164,0.176056,5.024421
12,0.784542,0.318757,0.209427,0.667144,1.913592
16,0.730284,0.572542,0.515,0.720704,2.704409


In [42]:
# Haversine distance is available when dimension is 2
df.filter(
    pds.query_radius_at_pt(
        pl.col("val1"), pl.col("val2"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5],
        r = 10, # in km
        dist = "h" 
    )
).head()

id,val1,val2,val3,r,rh
u64,f64,f64,f64,f64,f64
3,0.487158,0.55844,0.830182,0.478289,5.362583
24,0.428474,0.505632,0.732928,0.203035,4.484122
151,0.520347,0.490913,0.106325,0.289022,7.540303
154,0.419851,0.515412,0.101285,0.270371,8.914162
157,0.468544,0.537907,0.290244,0.878936,4.333144


In [43]:
df.filter(
    pds.query_radius_at_pt(
        pl.col("val1"), pl.col("val2"), 
        pt = [0.5, 0.5],
        # radius can also be an existing column in the dataframe.
        r = pl.col("rh"), 
        dist = "h" 
    )
).head()

id,val1,val2,val3,r,rh
u64,f64,f64,f64,f64,f64
151,0.520347,0.490913,0.106325,0.289022,7.540303
166,0.475936,0.470189,0.440466,0.923375,5.310322
473,0.430929,0.481859,0.648585,0.425522,9.839513
504,0.528381,0.481837,0.360711,0.208123,8.704986
511,0.440361,0.499506,0.858631,0.465153,6.66935


In [44]:
friends = df.select(
    pl.col("id").cast(pl.UInt64),
    pds.query_radius_ptwise(
        # Columns used as the coordinates in n-d space
        pl.col("val1"), pl.col("val2"), 
        index=pl.col("id"),
        r = 0.05, 
        dist = "l2",
    ).alias("friends")
).with_columns(
    pl.col("friends").list.len().alias("count")
)
friends.head()

id,friends,count
u64,list[u64],u32
0,"[0, 900, … 1099]",189
1,"[1, 1154, … 961]",311
2,"[2, 1273, … 1681]",243
3,"[3, 1813, … 1713]",305
4,"[4, 1277, … 159]",311


# Simple Graph Queries

There is limited functionality in the Graph module currently. E.g. Only constant cost per edge.

Graph queries are very expensive.

In [45]:
friends.select(
    pl.col("friends").graph.eigen_centrality() # .arg_max()
).head()

friends
f64
0.000219
0.000808
0.000343
0.000906
0.00088


In [46]:
friends.select(
    # Shortest path to the node at index = 3 (id = 3, the 4th node in this case), assuming constant cost per edge
    pl.col("friends").graph.shortest_path(target = 3, cost=None, parallel=True).alias("path")
).head()

path
list[u64]
"[987, 655, 3]"
[3]
"[1350, 978, 3]"
[]
[3]


In [47]:
friends.select(
    # Almost every node can reach node 3, and the number is the number steps to reach it
    # This is a way faster way to filter results if you don't need the actual path
    pl.col("friends").graph.reachable(3)
)

friends
struct[2]
"{true,3}"
"{true,1}"
"{true,3}"
"{true,0}"
"{true,1}"
…
"{true,2}"
"{true,3}"
"{true,2}"


In [48]:
relationships = pl.DataFrame({
    "id": range(5),
    "connections":[[1,2,3,4], [2,3], [4], [0,1,2], [1]],
    # Small values means closer
    "close-ness":[[0.4, 0.3, 0.2, 0.1], [0.1, 1], [0.5], [0.1, 0.1, 0.1], [0.1]]
}).with_columns(
    pl.col("connections").list.eval(pl.element().cast(pl.UInt64))
)
relationships.head()

id,connections,close-ness
i64,list[u64],list[f64]
0,"[1, 2, … 4]","[0.4, 0.3, … 0.1]"
1,"[2, 3]","[0.1, 1.0]"
2,[4],[0.5]
3,"[0, 1, 2]","[0.1, 0.1, 0.1]"
4,[1],[0.1]


In [49]:
# To go to node at id = 1, node at index 0 would rather go to 4 first and then 1.
relationships.select(
    pl.col("connections").graph.shortest_path(
        target = 1,
        cost = pl.col("close-ness"),
        parallel = False
    ).alias("path")
).unnest("path").head()

path,cost
list[u64],f64
"[4, 1]",0.2
[],0.0
"[4, 1]",0.6
[1],0.1
[1],0.1


In [50]:
# In and out deg
relationships.select(
    pl.col("connections").graph.in_out_deg().alias("deg")
).unnest("deg")

in_deg,out_deg
u32,u32
1,4
3,2
3,1
2,3
2,1


# String Nearest Neighbors

This might be very slow for very large vocab / column.

In [51]:
df = pl.DataFrame({
    "a":["AAAAA", "ABCABC", "AAAADDD", "ADSDSDS", "WORD"],
    "b":["AAAAT", "ABCACD", "ADSSD", "APPLES", "WORLD"] 
})

In [52]:
# Use Levenshtein to find the nearest neighbor in vocab to word in column a
df.select(
    pl.col("a").str2.similar_words(
        vocab = pl.col("b"),
        k = 1,
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
str
"""AAAAT"""
"""ABCACD"""
"""AAAAT"""
"""ADSSD"""
"""WORLD"""


In [53]:
# Use Levenshtein to find 2 nearest neighbors
df.select(
    pl.col("a").str2.similar_words(
        vocab = pl.col("b"),
        k = 2,
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
list[str]
"[""AAAAT"", ""ADSSD""]"
"[""ABCACD"", ""AAAAT""]"
"[""AAAAT"", ""ABCACD""]"
"[""ADSSD"", ""APPLES""]"
"[""WORLD"", ""ADSSD""]"


In [54]:
# Currently only Levenshtein and hamming are implemented for this
# Empty means nothing in vocab can be compared in the hamming sense with the corresponding word in a
df.select(
    pl.col("a").str2.similar_words(
        vocab = pl.col("b"),
        k = 2,
        threshold = 4, # <= threshold hamming distance away
        metric = "hamming"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
list[str]
"[""AAAAT"", ""ADSSD""]"
"[""ABCACD""]"
[]
[]
[]


In [55]:
# You may provide a vocab like this
df.select(
    pl.col("a"),
    pl.col("a").str2.similar_words(
        vocab = ["WORLD", "AAAAA", "ABCDEFG", "ZIV", "TQQQ"],
        k = 3,
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

a,similar_words_from_vocab
str,list[str]
"""AAAAA""","[""AAAAA"", ""ZIV"", ""WORLD""]"
"""ABCABC""","[""ABCDEFG"", ""AAAAA"", ""ZIV""]"
"""AAAADDD""","[""AAAAA"", ""WORLD"", ""ABCDEFG""]"
"""ADSDSDS""","[""ABCDEFG"", ""WORLD"", ""AAAAA""]"
"""WORD""","[""WORLD"", ""ZIV"", ""TQQQ""]"
